In [2]:
#!/usr/bin/env python3
# code to replicate the performance of the NIPS'16 MLHC workshop paper "Cam CNNS predict anatomy?"

from __future__ import absolute_import
from __future__ import print_function

import os
import csv
import six

import numpy as np
import time
import json
import warnings

from collections import deque
from collections import OrderedDict
from collections import Iterable

# os.environ["THEANO_FLAGS"] = "device=gpu0,floatX=float32" 

os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ[
#     "THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32,force_device=true"#,lib.cnmem=0.7 ,nvcc.flags=-D_FORCE_INLINES"
import sys
import numpy as np
from keras import backend as K
K.set_image_data_format('channels_first')
#from keras.datasets import cifar10
#keras.callbacks.Callback()
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Convolution2D, MaxPooling2D 
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback
from keras.models import load_model
from keras.datasets import mnist
from scipy import io
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
# from Ipython.display import display, clear_output


In [15]:
#!/usr/bin/env python3
# code to replicate the performance of the NIPS'16 MLHC workshop paper "Cam CNNS predict anatomy?"

from __future__ import absolute_import
from __future__ import print_function

import os
import csv
import six

import numpy as np
import time
import json
import warnings

from collections import deque
from collections import OrderedDict
from collections import Iterable

# os.environ["THEANO_FLAGS"] = "device=gpu0,floatX=float32" 

os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ[
#     "THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32,force_device=true"#,lib.cnmem=0.7 ,nvcc.flags=-D_FORCE_INLINES"
import sys
import numpy as np
from keras import backend as K
K.set_image_data_format('channels_first')
#from keras.datasets import cifar10
#keras.callbacks.Callback()
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Convolution2D, MaxPooling2D 
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback
from keras.models import load_model
from keras.datasets import mnist
from scipy import io
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
# from Ipython.display import display, clear_output


def load_data():
    X = np.load('/work/vsankar/Project-Luna/image_array.npy')
    Y = np.load('/work/vsankar/Project-Luna/new_labels.npy')
#     Tracer()()

    n=0;
    for i,y in enumerate(Y):

        if y<=2:
            Y[i] = 0.0            
        elif y>=4:
            Y[i] = 1.0
            
    y_true = map(lambda x:(x==0)|(x==1),Y)
    Y = Y[y_true]
    X = X[y_true]    
    
#     Tracer()()
    
    Yzero = Y[np.where(Y==0)]
    Xzero = X[np.where(Y==0)]

    Yone = Y[np.where(Y==1)]
    Xone = X[np.where(Y==1)]
    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
    #     shear_range = 0.34,  # value in radians, equivalent to 20 deg
    #     zoom_range = [1/1.6, 1.6],   #same as in NIPS 2015 paper.
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    n1 = 3
    #increase zero dataset
#     n1 = len(Yone)/len(Yzero)
    

    XzeroRotated , YzeroMRotated = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
    # Tracer()()
    for i in range(n1-2):
        XzeroM , YzeroM = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
        XzeroRotated = np.append(XzeroRotated,XzeroM,axis=0)
        YzeroMRotated = np.append(YzeroMRotated,YzeroM,axis=0)

    #increase one dataset
#     n1 = len(Yone)/len(Yzero)  
    XoneRotated , YoneMRotated = datagen.flow(Xone, Yone, batch_size=len(Yzero)).next()
    # Tracer()()
    for i in range(n1-2):
        XoneM , YoneM = datagen.flow(Xone, Yone, batch_size=len(Yone)).next()
        XoneRotated = np.append(XoneRotated,XoneM,axis=0)
        YoneMRotated = np.append(YoneMRotated,YoneM,axis=0)



    X = np.append(X,XzeroRotated,axis=0)
    Y = np.append(Y,YzeroMRotated,axis=0)

    X = np.append(X,XoneRotated,axis=0)
    Y = np.append(Y,YoneMRotated,axis=0)
    
#     Tracer()()
    
    n = len(Y)/10
    
    a = range(len(Y))
    np.random.shuffle(a)
    idx_test = a[:n]
    idx_val = a[n:n*2]
    idx_train = a[n*2:]
    
    X_train = X[idx_train]
    X_test = X[idx_test]
    X_val = X[idx_val]
    
    y_train = Y[idx_train]
    y_test = Y[idx_test]
    y_val = Y[idx_val]

    return X_train,y_train,X_test,y_test,X_val,y_val

def normalize_date(X_train,Y_train,X_test,Y_test,X_val, Y_val,nb_classes):
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    print(X_val.shape[0], 'val samples')
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    
    Y_train = Y_train.astype('float32')
    Y_test = Y_test.astype('float32')
    Y_val = Y_val.astype('float32')

#     normalizing the data
#     X_train /= 4095.0   
#     X_test /= 4095.0
#     X_val /= 4095.0
    
    #std
#     X_train = X_train/np.std(X_train) - np.mean(X_train)
#     X_test = X_test/np.std(X_test) - np.mean(X_test)
    
#     Tracer()()
    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(Y_train, nb_classes)
    Y_test = np_utils.to_categorical(Y_test, nb_classes)
    Y_val = np_utils.to_categorical(Y_val, nb_classes)
    
    
    return X_train,Y_train,X_test,Y_test,X_val,Y_val

In [16]:
X_train,Y_train,X_test,Y_test,X_val,Y_val  = load_data()
nb_classe=2
# X_train,Y_train,X_test,Y_test,X_val,Y_val = normalize_date(X_train,y_train,X_test,y_test,X_val,y_val,nb_classe)

In [9]:
print('y=1')
print(len(np.where(Y_train[:,1]==1)[0]))

print('y=0')
print(len(np.where(Y_train[:,1]==0)[0]))

y=1
18410
y=0
5573


In [10]:
Yzero = Y_train[np.where(Y_train[:,1]==0)]
Xzero = X_train[np.where(Y_train[:,1]==0)]

Yone = Y_train[np.where(Y_train[:,1]==1)]
Xone = X_train[np.where(Y_train[:,1]==1)]
# this will do preprocessing and realtime data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
#     shear_range = 0.34,  # value in radians, equivalent to 20 deg
#     zoom_range = [1/1.6, 1.6],   #same as in NIPS 2015 paper.
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

#increase zero dataset
n1 = len(Yone)/len(Yzero)
# n1 = 2

XzeroRotated , YzeroMRotated = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
# Tracer()()
for i in range(n1-2):
    XzeroM , YzeroM = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
    XzeroRotated = np.append(XzeroRotated,XzeroM,axis=0)
    YzeroMRotated = np.append(YzeroMRotated,YzeroM,axis=0)
    
#increase one dataset
# n1 = len(Yone)/len(Yzero)
# n_increase_zero = 2
# XoneRotated , YoneMRotated = datagen.flow(Xone, Yone, batch_size=len(Yzero)).next()
# # Tracer()()
# for i in range(n1-2):
#     XoneM , YoneM = datagen.flow(Xzero, Yzero, batch_size=len(Yzero)).next()
#     XoneRotated = np.append(XoneRotated,XoneM,axis=0)
#     YoneMRotated = np.append(YoneMRotated,YoneM,axis=0)



X_train = np.append(X_train,XzeroRotated,axis=0)
Y_train = np.append(Y_train,YzeroMRotated,axis=0)

In [18]:
print('y=1 train')
print(len(np.where(Y_train==1)[0]))

print('y=0 train')
print(len(np.where(Y_train==0)[0]))

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')

y=1 train
20917
y=0 train
16819
37736 train samples
4716 test samples
4716 val samples


In [19]:
print('X_train shape:', X_train.shape)


X_train shape: (37736, 1, 96, 96)


In [20]:
np.savez('/work/vsankar/Project-Luna/Train_nf_2_4_rotated_data',X_train =  X_train,Y_train = Y_train)
np.savez('/work/vsankar/Project-Luna/Test_nf_2_4_rotated_data', X_test=X_test,Y_test=Y_test)
np.savez('/work/vsankar/Project-Luna/Val_nf_2_4_rotated_data', X_val=X_val,Y_val=Y_val)

In [3]:
 data = np.load('/work/vsankar/Project-Luna/Train_nf_data.npz')

In [4]:
data['X_train']

array([[[[  946.,   969.,   950., ...,    81.,    82.,    70.],
         [  890.,   905.,   891., ...,    89.,    99.,    84.],
         [  909.,   900.,   890., ...,   100.,   108.,    93.],
         ..., 
         [ 1098.,  1093.,  1040., ...,   625.,   271.,   104.],
         [ 1059.,  1098.,  1079., ...,   446.,   156.,    78.],
         [ 1062.,  1044.,  1081., ...,   278.,   117.,   110.]]],


       [[[ 1042.,  1052.,  1084., ...,  1078.,  1047.,  1034.],
         [ 1030.,  1042.,  1070., ...,  1022.,   990.,   999.],
         [ 1029.,  1028.,  1046., ...,   915.,   912.,   937.],
         ..., 
         [  149.,   171.,   190., ...,   996.,   989.,   995.],
         [  164.,   180.,   198., ...,   938.,   943.,   940.],
         [  144.,   146.,   173., ...,   918.,   974.,   980.]]],


       [[[  202.,   205.,   171., ...,    74.,    89.,    96.],
         [  112.,   148.,   160., ...,    83.,    89.,    89.],
         [  101.,   100.,   109., ...,    89.,    92.,    91.],
  

In [ ]:
X_train,Y_train = np.load('/work/vsankar/Project-Luna/Train_nf_data.npy')
X_test,Y_test = np.load('/work/vsankar/Project-Luna/Test_nf_data.npy')
X_val,Y_val = np.load('/work/vsankar/Project-Luna/Val_nf_data.npy')

In [ ]:
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')

In [ ]:
 np.load('/work/vsankar/Project-Luna/Train_nf_data.npy')

In [ ]:
len(np.where(y_train==1)[0])

In [ ]:
type(YzeroM)

In [ ]:
0.1*96
